In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("Social_Network_ads.csv")
print("Data imported")

Data imported


In [3]:
df.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [4]:
X = df.iloc[:,[2,3]].values
X.shape

(400, 2)

In [5]:
y = df.iloc[:,-1].values
y.shape

(400,)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)
print("Train, test sets obtained!")

Train, test sets obtained!


In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

C:\Users\dhruv.aggarwal\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\dhruv.aggarwal\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\dhruv.aggarwal\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [8]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators= 10, criterion= 'entropy', random_state=0 )
#Randomly selecting init parameters

In [9]:
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [10]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [11]:
est = RandomForestClassifier(n_jobs=-1)
rf_p_dist={'max_depth':[3,5,10,None],
              'n_estimators':[10,100,200,300,400,500],
              'max_features':randint(1,3),
               'criterion':['gini','entropy'],
               'bootstrap':[True,False],
               'min_samples_leaf':randint(1,4),
}

In [12]:
def hypertuning_rscv(est, p_distr, nbr_iter,X,y):
    rdmsearch = RandomizedSearchCV(est, param_distributions=p_distr,
                                  n_jobs=-1, n_iter=nbr_iter, cv=9)
    #CV = Cross-Validation ( here using Stratified KFold CV)
    rdmsearch.fit(X,y)
    ht_params = rdmsearch.best_params_
    ht_score = rdmsearch.best_score_
    print('ht_params',ht_params)
    print('ht_score',ht_score)
    #return ht_params, ht_score

In [13]:
rf_parameters, rf_ht_score = print(hypertuning_rscv(est, rf_p_dist, 40, X, y))

C:\Users\dhruv.aggarwal\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


ht_params {'bootstrap': True, 'criterion': 'entropy', 'max_depth': 10, 'max_features': 2, 'min_samples_leaf': 3, 'n_estimators': 400}
ht_score 0.91
None


TypeError: 'NoneType' object is not iterable

In [ ]:
#using the above result to train a new clf

classifier=RandomForestClassifier( bootstrap = True, 
                                   criterion =  'gini' , 
                                   max_depth = 3, 
                                   max_features = 2, 
                                   min_samples_leaf = 3, 
                                   n_estimators = 100,
                                 n_jobs=-1)

In [ ]:
classifier.fit(X,y)
# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [ ]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(y_test, y_pred)
print('cm',cm)

accuracy_score=accuracy_score(y_test,y_pred)
print('accuracy_score',accuracy_score)
## Cross Validation good for selecting models
from sklearn.model_selection import cross_val_score

cross_val=cross_val_score(classifier,X,y,cv=10,scoring='accuracy').mean()
print('cross_val',cross_val)

**GridSearchCV** 
- works best for small no. of hyperparameters and when all the hyperparameters are of same magnitude

**RandomisedSearchCV**
- works best when magnitudes of influence are imbalanced which happens when no. of hyperparameters is growing